## Data transformation and Feature Engineering

In [1]:
import os
os.chdir('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system')

from pyspark.sql import SparkSession,Window
from pyspark.sql.types import StructField,StructType,StringType,IntegerType,FloatType,TimestampType
from pyspark.sql import functions as F
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
#  creat spark session

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("ProductRecommendationSystem") \
    .getOrCreate()

24/12/19 15:49:50 WARN Utils: Your hostname, oviemunooboro2 resolves to a loopback address: 127.0.0.1; using 10.0.0.4 instead (on interface eth0)
24/12/19 15:49:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/19 15:49:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([
    StructField('event_time',TimestampType(),False),
    StructField('event_type',StringType(),True),
    StructField('product_id',StringType(),True),
    StructField('category_id',StringType(),True),
    StructField('category_code',StringType(),True),
    StructField('brand',StringType(),True),
    StructField('price',FloatType(),True),
    StructField('user_id',StringType(),True),
    StructField('user_session',StringType(),True)]
)

In [4]:
oct_data = spark.read.csv('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system/Artifacts/ingested_data/2019-Oct.csv',header=True,schema=schema)
nov_data = spark.read.csv('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system/Artifacts/ingested_data/2019-Nov.csv',header=True,schema=schema)
dec_data = spark.read.csv('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system/Artifacts/ingested_data/2019-Dec.csv',header=True,schema=schema)
jan_data = spark.read.csv('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system/Artifacts/ingested_data/2020-Jan.csv',header=True,schema=schema)
feb_data = spark.read.csv('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system/Artifacts/ingested_data/2020-feb.csv',header=True,schema=schema)

In [5]:
oct_data.show()

+-------------------+----------------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|         event_time|      event_type|product_id|        category_id|category_code|   brand|price|  user_id|        user_session|
+-------------------+----------------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|2019-10-01 00:00:00|            cart|   5773203|1487580005134238553|         NULL|  runail| 2.62|463240011|26dd6e6e-4dac-477...|
|2019-10-01 00:00:03|            cart|   5773353|1487580005134238553|         NULL|  runail| 2.62|463240011|26dd6e6e-4dac-477...|
|2019-10-01 00:00:07|            cart|   5881589|2151191071051219817|         NULL|  lovely|13.48|429681830|49e8d843-adf3-428...|
|2019-10-01 00:00:07|            cart|   5723490|1487580005134238553|         NULL|  runail| 2.62|463240011|26dd6e6e-4dac-477...|
|2019-10-01 00:00:15|            cart|   5881449|1487580013522845895|         NULL|  lovel

In [6]:
datasets = [oct_data,nov_data,dec_data,jan_data,feb_data]

for data in datasets:
    data.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (

In [7]:
# split into training , validation and test data

training_data = oct_data.union(nov_data).union(dec_data)
validation_data = jan_data
testing_data = feb_data

In [8]:
# check to see if the user_id are both traingin,validation and test data

train_user_id = training_data.select('user_id').distinct()
validation_user_id = validation_data.select('user_id').distinct()
test_user_id = testing_data.select('user_id').distinct()


print(f'total number of users in training data {train_user_id.count()}')
print(f'total number of users in validation data {validation_user_id.count()}')
print(f'total number of users in test data {test_user_id.count()}')

total number of users in training data 1012561


total number of users in validation data 410073


total number of users in test data 391055


In [9]:
# Users in Validation but not in Training:

val_not_in_train = validation_user_id.subtract(train_user_id)
print(f"Number of users in validation not in training: {val_not_in_train.count()}")

Number of users in validation not in training: 328938


In [10]:
val_not_in_test = test_user_id.subtract(validation_user_id)
print(f"Number of users in test not in validation: {val_not_in_test.count()}")


Number of users in test not in validation: 327020


In [11]:
test_not_in_train = test_user_id.subtract(train_user_id)
print(f"Number of users in test not in train: {test_not_in_train.count()}")

Number of users in test not in train: 331069


There is an issue of cold start in the data given that majority of the user in the validation and test data are not present in the training data

In [12]:
train_product_id = training_data.select('product_id').distinct()
val_product_id = validation_data.select('product_id').distinct()
test_product_id = testing_data.select('product_id').distinct()


print(f'total number of product in training data {train_product_id.count()}')
print(f'total number of product in validation data {val_product_id.count()}')
print(f'total number of product in test data {test_product_id.count()}')


total number of product in training data 49341


total number of product in validation data 45484


total number of product in test data 48579


In [13]:
val_not_in_train = val_product_id.subtract(train_product_id)
print(f"Number of product in validation not in training: {val_not_in_train.count()}")

Number of product in validation not in training: 2087


In [14]:
val_not_in_test = val_product_id.subtract(test_product_id)
print(f"Number of product in validation not in test: {val_not_in_test.count()}")

Number of product in validation not in test: 2345


In [15]:
test_not_in_train = test_product_id.subtract(train_product_id)
print(f"Number of product in test not in train: {test_not_in_train.count()}")

Number of product in test not in train: 4623


In [16]:
# check missing values 
train_missing_values = training_data.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in training_data.columns])
val_missing_values = validation_data.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in validation_data.columns])


train_missing_values.show() , val_missing_values.show()

+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|  brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+
|         0|         0|         0|          0|     12069716|5155579|    0|      0|        2229|
+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+



+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|  brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+
|         0|         0|         0|          0|      4190033|1775630|    0|      0|        1314|
+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+



(None, None)

In [17]:
# making a copy of the training , val dataframe and caching into memory before begining preprocessing

new_training_data = training_data.select('*')
new_val_data = validation_data.select('*')

# cache into memory

new_training_data = new_training_data.cache()
new_val_data = new_val_data.cache()



####  handling the missing values

In [18]:
# handling the missing values

new_training_data = new_training_data.drop('category_code')
new_val_data = new_val_data.drop('category_code')


new_training_data = new_training_data.fillna({'brand':'UNKNOWN'})
new_val_data = new_val_data.fillna({'brand':'UNKNOWN'})

# Add the 'is_brand_missing' column to indicate if the brand is missing
new_training_data = new_training_data.withColumn('is_brand_missing', F.when(F.col('brand') == 'UNKNOWN',F.lit(1)).otherwise(F.lit(0)))

new_training_data = new_training_data.dropna(subset=['user_session'])
new_val_data = new_val_data.dropna(subset=['user_session'])

In [19]:
# check

train_missing_values = new_training_data.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in new_training_data.columns])
val_missing_values = new_val_data.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in new_val_data.columns])

train_missing_values.show() ,  val_missing_values.show()

24/12/19 15:53:19 WARN MemoryStore: Not enough space to cache rdd_257_12 in memory! (computed 14.1 MiB so far)================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================>                                                                                                                                                                                                                                                                                                            (9 + 4) / 13]

+----------+----------+----------+-----------+-----+-----+-------+------------+----------------+
|event_time|event_type|product_id|category_id|brand|price|user_id|user_session|is_brand_missing|
+----------+----------+----------+-----------+-----+-----+-------+------------+----------------+
|         0|         0|         0|          0|    0|    0|      0|           0|               0|
+----------+----------+----------+-----------+-----+-----+-------+------------+----------------+



+----------+----------+----------+-----------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-----+-----+-------+------------+
|         0|         0|         0|          0|    0|    0|      0|           0|
+----------+----------+----------+-----------+-----+-----+-------+------------+



(None, None)

In [20]:
#### check for duplicates 

training_duplicates = new_training_data.count() - new_training_data.dropDuplicates().count()
val_duplicates = new_val_data.count() - new_val_data.dropDuplicates().count()


print(f'Number of duplicates rows in training data : {training_duplicates}')
print(f'Number of duplicates rows in val data : {val_duplicates}')

24/12/19 15:53:53 WARN MemoryStore: Not enough space to cache rdd_257_4 in memory! (computed 45.3 MiB so far)=====================================================================================================================================================================================================================================================================================>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        (5 + 4) / 13]

Number of duplicates rows in training data : 643522
Number of duplicates rows in val data : 224922


In [21]:
duplicates  =  new_training_data.groupBy(new_training_data.columns).count().filter("count > 1")
duplicates.show()

24/12/19 15:54:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       (0 + 4) / 13]

+-------------------+----------------+----------+-------------------+--------+-----+---------+--------------------+----------------+-----+
|         event_time|      event_type|product_id|        category_id|   brand|price|  user_id|        user_session|is_brand_missing|count|
+-------------------+----------------+----------+-------------------+--------+-----+---------+--------------------+----------------+-----+
|2019-10-01 00:18:44|remove_from_cart|   5857277|1487580005134238553|  runail| 2.62|446404021|abac4e23-d977-47d...|               0|    2|
|2019-10-01 01:16:05|remove_from_cart|   5861352|1487580009521479714| UNKNOWN| 3.65|550109931|1119c8b2-0b90-4fb...|               1|    2|
|2019-10-01 05:21:45|            cart|   5743974|1487580013053083824| italwax| 1.98|486637085|205bec28-f4a8-4c8...|               0|    3|
|2019-10-01 05:25:01|remove_from_cart|     36546|1487580007072007091|ingarden| 5.46|546705258|75d137b0-8ab4-50b...|               0|    2|
|2019-10-01 05:29:14|      

In [22]:
event_type_counts = duplicates.groupBy('event_type').count()

event_type_counts.show()

24/12/19 15:55:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       (0 + 4) / 13]

+----------------+------+
|      event_type| count|
+----------------+------+
|        purchase|   601|
|            view|   353|
|            cart| 54812|
|remove_from_cart|555921|
+----------------+------+



In [23]:
new_training_data = new_training_data.dropDuplicates()


In [24]:
new_val_data = new_val_data.dropDuplicates()

In [25]:
# check price column

price_data = new_training_data.filter(new_training_data['price'] < 0 )

price_data.show()
print(f' the number of rows with negative prices : {price_data.count()}')

24/12/19 15:55:48 WARN MemoryStore: Not enough space to cache rdd_257_7 in memory! (computed 13.3 MiB so far)
24/12/19 15:55:48 WARN MemoryStore: Not enough space to cache rdd_257_4 in memory! (computed 45.3 MiB so far)======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================>                                                                                                                                                                                                                                                                                      

+-------------------+----------+----------+-------------------+-------+------+---------+--------------------+----------------+
|         event_time|event_type|product_id|        category_id|  brand| price|  user_id|        user_session|is_brand_missing|
+-------------------+----------+----------+-------------------+-------+------+---------+--------------------+----------------+
|2019-10-02 08:30:03|  purchase|   5716855|1487580014042939619|UNKNOWN| -7.94|550375225|5ddec778-9464-451...|               1|
|2019-10-01 19:10:56|  purchase|   5716857|1487580014042939619|UNKNOWN|-23.81|552507528|dcdd60c6-1a70-442...|               1|
|2019-10-03 17:37:04|  purchase|   5716859|1487580014042939619|UNKNOWN|-47.62|555414763|479149eb-1807-417...|               1|
|2019-10-03 18:25:39|  purchase|   5670257|1487580014042939619|UNKNOWN|-15.87|556383221|4333d203-bc4d-4d0...|               1|
|2019-10-13 16:46:01|  purchase|   5716857|1487580014042939619|UNKNOWN|-23.81|559820267|f178c995-f004-404...|  

24/12/19 15:55:48 WARN MemoryStore: Not enough space to cache rdd_257_4 in memory! (computed 45.3 MiB so far)
24/12/19 15:55:48 WARN MemoryStore: Not enough space to cache rdd_257_7 in memory! (computed 13.3 MiB so far)


 the number of rows with negative prices : 53


In [26]:
# convert negative prices to positive values

new_training_data = new_training_data.withColumn('price',F.abs(new_training_data['price']))
new_val_data = new_val_data.withColumn('price',F.abs(new_val_data['price']))

In [27]:
price_data = new_training_data.filter(new_training_data['price'] < 0 )

price_data.show()
print(f' the number of rows with negative prices : {price_data.count()}')

24/12/19 15:55:50 WARN MemoryStore: Not enough space to cache rdd_257_4 in memory! (computed 7.0 MiB so far)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               (0 + 4) / 13]

+----------+----------+----------+-----------+-----+-----+-------+------------+----------------+
|event_time|event_type|product_id|category_id|brand|price|user_id|user_session|is_brand_missing|
+----------+----------+----------+-----------+-----+-----+-------+------------+----------------+
+----------+----------+----------+-----------+-----+-----+-------+------------+----------------+



24/12/19 15:55:52 WARN MemoryStore: Not enough space to cache rdd_257_6 in memory! (computed 13.5 MiB so far)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              (0 + 4) / 13]

 the number of rows with negative prices : 0


In [28]:
new_training_data.show()

24/12/19 15:55:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       (0 + 4) / 13]

+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+
|         event_time|      event_type|product_id|        category_id|  brand|price|  user_id|        user_session|is_brand_missing|
+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+
|2019-10-01 00:52:00|            view|   5664161|1487580009445982239|UNKNOWN| 5.16|553602927|4eecd233-e925-414...|               1|
|2019-10-01 00:57:40|            cart|   5841842|1487580007675986893|   milv| 1.59|532551564|d7296e7c-5d83-edc...|               0|
|2019-10-01 01:46:07|            view|   5849719|1783999063574708423|bluesky| 3.97|542286071|cc3a724d-7f50-409...|               0|
|2019-10-01 01:55:43|            cart|   5757469|1487580008246412266| matrix| 9.52|553103073|3072288c-6aeb-40e...|               0|
|2019-10-01 02:12:29|            view|   5692047|1487580007835370453|staleks

## create user features

In [29]:
# purchases made in the last 2 days

train_purchase = new_training_data.filter(new_training_data['event_type'] == 'purchase') 

train_purchase_per_2day = train_purchase.withColumn('two_day_purchase', 
                                                             F.from_unixtime(
                                                                 F.floor(F.unix_timestamp(F.col('event_time')) / (2 * 86400))* (2 * 86400) 
                                                                 )
                                                             )

train_purchase_per_2day = train_purchase_per_2day.groupBy('user_id','two_day_purchase').agg(F.count('*').alias('Num_purchase_per_2day'))
train_purchase_per_2day = train_purchase_per_2day.withColumnRenamed("user_id", "train_user_id") # rename 'user_id to prevent ambitiuity

train_purchase_per_2day.show()

24/12/19 15:56:18 WARN MemoryStore: Not enough space to cache rdd_257_6 in memory! (computed 6.9 MiB so far)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               (0 + 4) / 13]

+-------------+-------------------+---------------------+
|train_user_id|   two_day_purchase|Num_purchase_per_2day|
+-------------+-------------------+---------------------+
|    443356058|2019-10-01 00:00:00|                    8|
|    556271034|2019-10-03 00:00:00|                   14|
|    333709300|2019-10-01 00:00:00|                   30|
|    528207430|2019-10-01 00:00:00|                   11|
|    556700214|2019-10-03 00:00:00|                    4|
|    555790106|2019-10-07 00:00:00|                   17|
|    556924312|2019-10-05 00:00:00|                    5|
|    556983721|2019-10-05 00:00:00|                    7|
|    552912036|2019-10-03 00:00:00|                   11|
|    395968291|2019-10-05 00:00:00|                   13|
|    555240300|2019-10-01 00:00:00|                   11|
|    445702327|2019-10-01 00:00:00|                   19|
|    448666320|2019-10-05 00:00:00|                    7|
|    450686495|2019-10-03 00:00:00|                   15|
|    289331807

In [30]:
# Rename user_id in train_purchase_per_2day to avoid duplication in final result
train_purchase_per_2day = train_purchase_per_2day.withColumnRenamed("user_id", "train_user_id")

new_training_data = new_training_data.join(
    train_purchase_per_2day,
    (new_training_data['user_id'] == train_purchase_per_2day['train_user_id']) & 
    (F.floor(F.unix_timestamp(new_training_data['event_time']) / (2*86400)) ==  F.floor(F.unix_timestamp(train_purchase_per_2day['two_day_purchase']) / (2 * 86400))),
    how='left'
).drop('two_day_purchase','train_user_id')

# filling missing missin values in the num_purchase_per2_day column with 0 meaning no purchase where made in the 2 day interval
new_training_data = new_training_data.na.fill({'Num_purchase_per_2day' : 0 })

new_training_data.show()


24/12/19 15:56:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                                                                                                                                                                                                   (0 + 4) / 13][Stage 197:>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           (0 + 0) / 13]

+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+
|         event_time|      event_type|product_id|        category_id|  brand|price|  user_id|        user_session|is_brand_missing|Num_purchase_per_2day|
+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+
|2019-10-01 02:19:17|            cart|   5706784|1487580005092295511| runail| 4.44|550154276|c937f69d-67c9-483...|               0|                   15|
|2019-10-01 01:55:43|            cart|   5757469|1487580008246412266| matrix| 9.52|553103073|3072288c-6aeb-40e...|               0|                    0|
|2019-10-01 02:41:48|        purchase|   5723487|1487580005134238553| runail| 2.62|555452209|4ed108ec-1b02-4b4...|               0|                   64|
|2019-10-01 03:53:45|            cart|   5804167|1487580005671109489| masura

In [31]:
# average purchase price 

avg_purchase_price = train_purchase.groupBy('user_id').agg(F.format_number(F.mean('price'),2).cast(FloatType()).alias('Avg_purchase_price'))
avg_purchase_price = avg_purchase_price.withColumnRenamed("user_id", "train_user_id")

new_training_data = new_training_data.join(
    avg_purchase_price,
    new_training_data['user_id'] == avg_purchase_price['train_user_id'],
    how='left'
).drop('train_user_id')

new_training_data = new_training_data.na.fill({'Avg_purchase_price' : 0})

new_training_data.show()

24/12/19 15:57:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 213:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 214:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]]

+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+------------------+
|         event_time|      event_type|product_id|        category_id|  brand|price|  user_id|        user_session|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|
+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+------------------+
|2019-10-01 00:57:40|            cart|   5841842|1487580007675986893|   milv| 1.59|532551564|d7296e7c-5d83-edc...|               0|                    0|               0.0|
|2019-10-01 04:40:00|            view|   5818205|1487580013690618064|UNKNOWN| 35.3|542024414|c82d08ab-1e7f-4f7...|               1|                    0|               0.0|
|2019-10-01 02:19:17|            cart|   5706784|1487580005092295511| runail| 4.44|550154276|c937f69d-67c9-483...|               0|    

In [32]:
# most purchase category of user 

# groupby user id and brand and create a window specification to get the top three brands
brand_count = train_purchase.groupBy('user_id','brand').agg(F.count('brand').alias('brand_count'))

window_spec = Window.partitionBy('User_id').orderBy(F.desc('brand_count'))
ranked_brands = brand_count.withColumn('rank' ,F.row_number().over(window_spec))
top_brands = ranked_brands.filter(F.col("rank") <= 3)

top_brands = top_brands.groupBy("user_id").pivot("rank", [1, 2, 3]).agg(F.first("brand"))

# rename the columns
top_brands = top_brands.select(
    F.col("user_id"),
    F.col("1").alias("1_most_purchased_brand"),
    F.col("2").alias("2_most_purchased_brand"),
    F.col("3").alias("3_most_purchased_brand")
)

# Get the most purchased brand for each user
most_purchased_brand = brand_count.withColumn('rank', F.row_number().over(Window.partitionBy('user_id').orderBy(F.desc('brand_count'))))
most_purchased_brand = most_purchased_brand.filter(F.col('rank') == 1).select('user_id', 'brand')

# Join the top brands with the most purchased brand
top_brands_with_most_purchased = top_brands.join(most_purchased_brand, on='user_id', how='left')
top_brands_with_most_purchased = top_brands_with_most_purchased.withColumnsRenamed({'user_id':'train_user_id','brand':'preferred_brand'})
# Fill missing values with the user's most purchased brand
# Replace missing values with the preferred brand
top_brands_with_most_purchased = top_brands_with_most_purchased.withColumn(
    '1_most_purchased_brand',
    F.when(F.col('1_most_purchased_brand').isNull(), F.col('preferred_brand')).otherwise(F.col('1_most_purchased_brand'))
).withColumn(
    '2_most_purchased_brand',
    F.when(F.col('2_most_purchased_brand').isNull(), F.col('preferred_brand')).otherwise(F.col('2_most_purchased_brand'))
).withColumn(
    '3_most_purchased_brand',
    F.when(F.col('3_most_purchased_brand').isNull(), F.col('preferred_brand')).otherwise(F.col('3_most_purchased_brand'))
)



top_brands_with_most_purchased.show()

new_training_data = new_training_data.join(
    top_brands_with_most_purchased,
    new_training_data['user_id'] == top_brands_with_most_purchased['train_user_id'],
    how = 'left'
).drop('train_user_id','preferred_brand')

24/12/19 15:57:56 WARN MemoryStore: Not enough space to cache rdd_257_4 in memory! (computed 45.3 MiB so far)================================================>                                                                                                                                                                                                                                                                                                                                         (4 + 4) / 13][Stage 244:>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           (0 + 0) / 13]

+-------------+----------------------+----------------------+----------------------+---------------+
|train_user_id|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|preferred_brand|
+-------------+----------------------+----------------------+----------------------+---------------+
|    100787781|               UNKNOWN|               markell|                 estel|        UNKNOWN|
|     10079204|                kaaral|                   cnd|                kaaral|         kaaral|
|    101025416|                 irisk|               UNKNOWN|                   cnd|          irisk|
|    103274988|             bpw.style|               UNKNOWN|             freedecor|      bpw.style|
|    103540490|               UNKNOWN|               UNKNOWN|               UNKNOWN|        UNKNOWN|
|    105075440|              skinlite|               UNKNOWN|              estelare|       skinlite|
|    107113173|              farmstay|              glysolid|              skinlite|       

In [33]:
new_training_data.show()

24/12/19 15:58:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 264:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 265:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]]

24/12/19 15:58:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 15:58:11 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+------------------+----------------------+----------------------+----------------------+
|         event_time|      event_type|product_id|        category_id|  brand|price|  user_id|        user_session|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|
+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+------------------+----------------------+----------------------+----------------------+
|2019-10-01 00:57:40|            cart|   5841842|1487580007675986893|   milv| 1.59|532551564|d7296e7c-5d83-edc...|               0|                    0|               0.0|                  NULL|                  NULL|                  NULL|
|2019-10-01 04:40:00|           

In [ ]:
# replace missing values in purchased brands as unknown
new_training_data = new_training_data.na.fill({'1_most_purchased_brand':'UNKNOWN',
                                                   '2_most_purchased_brand':'UNKNOWN',
                                                   '3_most_purchased_brand' : 'UNKNOWN'})

new_training_data


DataFrame[event_time: timestamp, event_type: string, product_id: string, category_id: string, brand: string, price: float, user_id: string, user_session: string, is_brand_missing: int, Num_purchase_per_2day: bigint, Avg_purchase_price: float, 1_most_purchased_brand: string, 2_most_purchased_brand: string, 3_most_purchased_brand: string]

In [35]:
new_training_data.show()

24/12/19 15:59:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 317:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 318:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]]

+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+------------------+----------------------+----------------------+----------------------+
|         event_time|      event_type|product_id|        category_id|  brand|price|  user_id|        user_session|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|
+-------------------+----------------+----------+-------------------+-------+-----+---------+--------------------+----------------+---------------------+------------------+----------------------+----------------------+----------------------+
|2019-10-01 00:57:40|            cart|   5841842|1487580007675986893|   milv| 1.59|532551564|d7296e7c-5d83-edc...|               0|                    0|               0.0|               UNKNOWN|               UNKNOWN|               UNKNOWN|
|2019-10-01 04:40:00|           

### session based features 

In [36]:
# duration of users session

session_duration = new_training_data.groupBy('user_id','user_session').agg(
    F.min(F.col('event_time')).alias('session_start'),
    F.max(F.col('event_time')).alias('session_end')
).withColumn('session_duration', F.unix_timestamp('session_end') - F.unix_timestamp('session_start'))


session_duration.show()

# combine 

new_training_data = new_training_data.join(
    session_duration,
    on = ['user_id','user_session'],
    how='left'

).drop('session_start','session_end')

24/12/19 16:00:06 WARN MemoryStore: Not enough space to cache rdd_257_4 in memory! (computed 45.3 MiB so far)====================================================>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         (2 + 4) / 13]

+---------+--------------------+-------------------+-------------------+----------------+
|  user_id|        user_session|      session_start|        session_end|session_duration|
+---------+--------------------+-------------------+-------------------+----------------+
|555451330|1da100d7-dd1d-4e5...|2019-10-01 00:36:57|2019-10-01 00:36:57|               0|
|458264120|bb72b357-2782-4d9...|2019-10-01 02:39:29|2019-10-01 02:39:29|               0|
|555482515|61fdb35c-409a-47e...|2019-10-01 04:23:00|2019-10-01 04:23:00|               0|
|324741429|9b57e28b-aaac-4b8...|2019-10-01 04:28:59|2019-10-01 04:28:59|               0|
|377119832|834f2167-3a25-4b0...|2019-10-01 04:42:11|2019-10-01 04:42:11|               0|
|444503648|527bb3d3-618a-400...|2019-10-01 04:45:16|2019-10-01 04:50:10|             294|
|380837147|a931d616-b6be-420...|2019-10-01 05:04:01|2019-10-01 05:13:13|             552|
|514703299|706b6a1e-7cd4-4fe...|2019-10-01 05:39:57|2019-10-01 05:39:57|               0|
|555497866

In [37]:
new_training_data.show()

24/12/19 16:00:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 373:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 374:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]


+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+
|  user_id|        user_session|         event_time|      event_type|product_id|        category_id|  brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|
+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+
|227060608|bf1a1eaf-eef0-414...|2019-10-01 03:34:19|remove_from_cart|   5561584|1487580007852147670|UNKNOWN| 5.48|               1|                    0|              3.08|               UNKNOWN|                 irisk|     

In [40]:
# total number of product viewed during a session

session_view = new_training_data.filter(F.col('event_type') == 'view')
num_session_view = session_view.groupBy('user_id','user_session').agg(F.count('event_type').alias('num_session_views'))

num_session_view.show()

# combine dataframe

new_training_data = new_training_data.join(
    num_session_view,
    on = ['user_id','user_session'],
    how='left'
)

new_training_data = new_training_data.na.fill({'num_session_views' : 0})


24/12/19 16:11:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 585:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 586:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]]

+---------+--------------------+-----------------+
|  user_id|        user_session|num_session_views|
+---------+--------------------+-----------------+
|100787781|188a44b5-83f1-4f1...|                4|
|100787781|9a95e186-1c1a-4ea...|                1|
|100787781|a91ab4bf-6d76-46f...|               10|
|100787781|ab88df75-5e32-419...|                6|
|100787781|33e757b9-11c7-4e8...|                4|
|100787781|53029ceb-b63b-4a9...|                3|
|100787781|4eb304f5-b558-4a0...|                3|
|101025416|bd5f70a4-fb1b-4ad...|                3|
|101025416|caba2c83-68dd-420...|               39|
|101025416|3506dd98-9567-48c...|                1|
|101025416|dd5a7463-a0be-4a2...|                4|
|101025416|bc3b1686-872a-486...|               21|
|101025416|4c6468ec-42fe-489...|                1|
|101025416|c4acfcb3-79fd-4f4...|                1|
|101245475|ed58d2ad-5e4d-49b...|                1|
|101245475|d771406b-a890-4ad...|                1|
|101245475|fe95eee1-4c1f-4a5...

In [41]:
new_training_data.show()

24/12/19 16:12:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 617:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 618:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]


+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+
|  user_id|        user_session|         event_time|      event_type|product_id|        category_id|  brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|num_session_views|
+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+
|227060608|bf1a1eaf-eef0-414...|2019-10-01 03:34:19|remove_from_cart|   5561584|1487580007852147670|UNKNOWN| 5.48|               1|                    0|              3.

In [46]:
# number of product added ti cart in a sesiion

session_cart = new_training_data.filter(F.col('event_type') == 'cart')
num_session_cart = session_cart.groupBy('user_id','user_session').agg(F.count('event_type').alias('num_session_cart'))

num_session_cart.show()

# combine dataframe

new_training_data = new_training_data.join(
    num_session_cart,
    on = ['user_id','user_session'],
    how='left'
)

new_training_data = new_training_data.na.fill({'num_session_cart' : 0})

24/12/19 16:32:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 1137:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13][Stage 1138:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13]]

+---------+--------------------+----------------+
|  user_id|        user_session|num_session_cart|
+---------+--------------------+----------------+
|100787781|53029ceb-b63b-4a9...|               4|
|100787781|188a44b5-83f1-4f1...|              10|
|100787781|a91ab4bf-6d76-46f...|               8|
|100787781|ab88df75-5e32-419...|               4|
| 10079204|f873d16b-4efa-4e9...|               2|
|101025416|4c6468ec-42fe-489...|              11|
|101025416|caba2c83-68dd-420...|              12|
|101025416|bc3b1686-872a-486...|               4|
|102771868|e7c88b67-5db3-4ec...|               6|
|102771868|9f8bfb11-afbf-4a6...|               1|
|102771868|e3b8573f-c32b-44f...|               3|
|102771868|35c79850-8b0f-4d8...|               2|
|103156342|d82f0914-e57e-457...|               6|
|103274988|09ac3d1b-aaf9-442...|              13|
|103274988|43063873-c7e8-4ac...|               5|
|103540490|13c6b5da-c6c1-4b8...|               8|
|103540490|70337e79-b8ce-47f...|               8|


In [ ]:
new_training_data.show()

24/12/19 16:28:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 978:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13][Stage 979:>                                                                                                                                                                                                                                                                                                                    (0 + 0) / 13]


+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+
|  user_id|        user_session|         event_time|      event_type|product_id|        category_id|  brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|num_session_views|num_session_cart|
+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+
|227060608|bf1a1eaf-eef0-414...|2019-10-01 03:34:19|remove_from_cart|   5561584|1487580007852147670|UNKNOWN| 5.48|    

In [51]:
# number of produt removed from cart

session_frm_cart = new_training_data.filter(F.col('event_type') == 'remove_from_cart')
num_session_frm_cart = session_frm_cart.groupBy('user_id','user_session').agg(F.count('event_type').alias('num_session_frm_cart'))

num_session_frm_cart.show()

# combine dataframe

new_training_data= new_training_data.join(
    num_session_frm_cart,
    on = ['user_id','user_session'],
    how='left'
)

new_training_data = new_training_data.na.fill({'num_session_frm_cart' : 0})


24/12/19 16:44:37 WARN MemoryStore: Not enough space to cache rdd_257_7 in memory! (computed 33.0 MiB so far)                                                                                                                                                                                                                   (2 + 4) / 13][Stage 1643:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13][Stage 1644:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13]]

+---------+--------------------+--------------------+
|  user_id|        user_session|num_session_frm_cart|
+---------+--------------------+--------------------+
|100787781|188a44b5-83f1-4f1...|                   5|
|100787781|ab88df75-5e32-419...|                   7|
|100787781|a91ab4bf-6d76-46f...|                   4|
|101025416|caba2c83-68dd-420...|                  15|
|102771868|a1b846c0-bad5-47b...|                   3|
|102771868|cc1b259a-3b77-469...|                   5|
|102771868|e3b8573f-c32b-44f...|                   2|
|103156342|d82f0914-e57e-457...|                   1|
|103274988|43063873-c7e8-4ac...|                   1|
|103274988|09ac3d1b-aaf9-442...|                   2|
|103540490|70337e79-b8ce-47f...|                  17|
|103540490|f11f9da4-ac38-4d5...|                   2|
|105075440|6e6853cf-491b-412...|                   4|
|105075440|d336bb7a-fcb0-4f3...|                   4|
|105075440|d0381610-7532-4ea...|                   2|
|108242662|2eb8f823-8c17-42f

In [52]:
new_training_data.show()

24/12/19 16:45:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 1675:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13][Stage 1676:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13]]

+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+--------------------+
|  user_id|        user_session|         event_time|      event_type|product_id|        category_id|  brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|num_session_views|num_session_cart|num_session_frm_cart|
+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+--------------------+
|227060608|bf1a1eaf-eef0-414...|2019-10-01 03:34:19|rem

In [54]:
# session purchase

session_purchase = new_training_data.filter(F.col('event_type') == 'purchase')
num_session_purchase = session_purchase.groupBy('user_id', 'user_session').agg(F.count('event_type').alias('num_session_purchase'))

new_training_data = new_training_data.join(
    num_session_purchase,
    on=['user_id', 'user_session'],
    how='left'
)

new_training_data = new_training_data.na.fill({'num_session_purchase':0})

new_training_data.show()

24/12/19 16:57:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 2125:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13][Stage 2126:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13]


[4079.229s][warning][gc,alloc] Executor task launch worker for task 11.0 in stage 2124.0 (TID 4543): Retried waiting for GCLocker too often allocating 524288 words


24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/19 16:57:47 WARN RowBasedKeyValueBatch: Calling spill() on

+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+--------------------+--------------------+
|  user_id|        user_session|         event_time|      event_type|product_id|        category_id|  brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|num_session_views|num_session_cart|num_session_frm_cart|num_session_purchase|
+---------+--------------------+-------------------+----------------+----------+-------------------+-------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+--------------------+--------------

In [57]:
# cart-to-purchase ration

new_training_data = new_training_data.withColumn('cart_to_purchase_ratio', F.when(F.col('num_session_purchase') == 0 , 0).otherwise(
    F.col('num_session_cart') / F.col('num_session_purchase')
))

new_training_data = new_training_data.withColumn('cart_to_purchase_ratio', F.round(F.col('cart_to_purchase_ratio'), 2))


In [58]:

# Calculate Cart Abandonment Rate and round to two decimal points
new_training_data = new_training_data.withColumn('cart_abandonment_rate', 
                           F.when(F.col('num_session_cart') == 0, 0)  # Handle case where no products were added to the cart
                           .otherwise(1 - (F.col('num_session_purchase') / F.col('num_session_cart'))))

new_training_data = new_training_data.withColumn('cart_abandonment_rate', F.round(F.col('cart_abandonment_rate'), 2))



In [59]:
new_training_data.show()

24/12/19 17:15:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 2796:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13][Stage 2797:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13]


+---------+--------------------+-------------------+----------+----------+-------------------+----------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+--------------------+--------------------+----------------------+---------------------+
|  user_id|        user_session|         event_time|event_type|product_id|        category_id|     brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|num_session_views|num_session_cart|num_session_frm_cart|num_session_purchase|cart_to_purchase_ratio|cart_abandonment_rate|
+---------+--------------------+-------------------+----------+----------+-------------------+----------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+------

### Product features 

In [63]:
product_purchase = train_purchase.groupBy('product_id').agg(
    F.count('product_id').alias('num_product_purchased')
)



result = new_training_data.join(
    product_purchase,
    on=['product_id'],
    how='left'
)

result.show()

24/12/19 17:47:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.                                                                                                                                                                                                            (0 + 4) / 13][Stage 3237:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13][Stage 3238:>                                                                                                                                                                                                                                                                                                                   (0 + 0) / 13]]

+----------+---------+--------------------+-------------------+----------+-------------------+----------+-----+----------------+---------------------+------------------+----------------------+----------------------+----------------------+----------------+-----------------+----------------+--------------------+--------------------+----------------------+---------------------+---------------------+
|product_id|  user_id|        user_session|         event_time|event_type|        category_id|     brand|price|is_brand_missing|Num_purchase_per_2day|Avg_purchase_price|1_most_purchased_brand|2_most_purchased_brand|3_most_purchased_brand|session_duration|num_session_views|num_session_cart|num_session_frm_cart|num_session_purchase|cart_to_purchase_ratio|cart_abandonment_rate|num_product_purchased|
+----------+---------+--------------------+-------------------+----------+-------------------+----------+-----+----------------+---------------------+------------------+----------------------+--------

In [ ]:
# NEED TO DO

# convert cart abondonment rate and cart rate to session
# add total product purchase instead
